In [ ]:
# times square parameters, for when you don't run in times square. Nothing in this cell will be executed in times square.
day_obs = "20250922"
n_nights_prev = 30

# Fault counts per subsystem - {{ params.day_obs }} and previous {{ params.n_nnights_prev }} nights

In [ ]:
import os
if os.getenv("EXTERNAL_INSTANCE_URL") is not None:
    print("updating rubin_nights")
    !pip install --user --upgrade git+https://github.com/lsst-sims/rubin_nights.git  --no-deps  > /dev/null 2>&1

In [ ]:
# Import minimum necessary packages
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
# For lots of colors use glasbey
from colorcet import glasbey 

from astropy.time import Time, TimeDelta
import astropy.units as u

from rubin_nights import connections, scriptqueue
import rubin_nights.dayobs_utils as rn_dayobs

if 'usdf' in os.getenv("EXTERNAL_INSTANCE_URL", ""):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# Hack for Lynne - but you can use your own token (outside of RSP)
if os.getenv("EXTERNAL_INSTANCE_URL") is None:
    tokenfile = '/Users/lynnej/.lsst/usdf_rsp'
    site = 'usdf'
    #tokenfile = '/Users/lynnej/.lsst/summit_rsp'
    #site = 'summit'
else:
    tokenfile = None
    site = None


In [ ]:
endpoints = connections.get_clients(tokenfile=tokenfile, site=site)

# Could run this over all days or over a single day .. or per day and then add new days .. 
t_start = Time('2025-09-01T12:00:00', format='isot', scale='utc')
t_end = Time('2025-09-22T12:00:00', format='isot', scale='utc')

error_topics = [t.replace('lsst.sal.', '').replace('.logevent_errorCode', '') for t in endpoints['efd'].get_topics() if 'errorCode' in t]

errors = scriptqueue.get_error_codes(t_start, t_end, endpoints['efd'])

errors['day_obs_mjd'] = np.zeros(len(errors), float)
errors['day_obs'] = np.zeros(len(errors), int)
errors['day'] = np.empty(len(errors), str)
def time_to_dayobs(x):
    x.day_obs_mjd = int(np.floor(Time(x.name, scale='utc').mjd - 0.5))
    x.day = Time(x.day_obs_mjd, format='mjd', scale='utc').iso[5:10]
    x.day_obs = Time(x.day_obs_mjd, format='mjd', scale='utc').iso[0:10].replace('-', '')
    return x
    
errors = errors.apply(time_to_dayobs, axis=1)


print(len(errors))

In [ ]:
# query = "select day_obs, visit_id, obs_start, obs_start_mjd, obs_end_mjd, band, science_program, img_type from cdb_lsstcam.visit1 where day_obs >= 20250415"
# visits = endpoints['consdb'].query(query)

# visits['time'] =  pd.to_datetime(visits.obs_start, format='mixed')
# visits.set_index('time', inplace=True)
# visits = visits.tz_localize('UTC')

# visits['salIndex'] = -10

# print(len(visits))

In [ ]:
groups = {}
groups['EnvSys'] = ['DIMM', 'DREAM', 'EAS', 'ESS', 'EPM', 'HVAC', 'OCPS', 'Watcher', 'ATBuilding',]
error_update = [t for t in error_topics if t not in groups['EnvSys']]
groups['CalSys'] = ['CBP', 'LaserTracker', 'Electrometer', 'FiberSpectrograph', 'LEDProjector', 'LinearStage', 'TunableLaser', 'PMD', 
                   'ATWhiteLight', 'ATMonochromator', 'ATSpectrograph']
error_update = [t for t in error_update if t not in groups['CalSys']]
groups['Simonyi'] = [t for t in error_update if t.startswith("MT")] + ['GIS', 'Scheduler', 'ScriptQueue', 'NewMTMount']
error_update = [t for t in error_update if t not in groups['Simonyi']]
groups['AuxTel'] = [t  for t in error_update if t.startswith("AT")] + ['Scheduler', 'ScriptQueue']
error_update = [t for t in error_update if t not in groups['AuxTel']]
groups['Other'] = error_update
#groups
addname = "CSC error"
for group in groups:
    updated_names = [t + addname for t in groups[group]]
    groups[group] = updated_names

In [ ]:
# Or create a pivot table, to set up a 'dashboard' style count per day per CSC
dayerrs = errors.groupby(['component', 'day']).agg({'component': 'count'})
dayerrs = dayerrs.reset_index('day')
dayerrs = dayerrs.pivot(columns=['day'], values='component')
dayerrs = dayerrs.fillna(0)


# We can limit the CSCs or dayobs displayed -- or sort
ecounts = errors.groupby('component').count()['error_code'].sort_values(ascending=False)
ecscs = list(ecounts.index)

# Simply no-flags display .. 
# dayerrs.loc[ecscs].iloc[:, last_n:]

# but we could also do some work with the pivot table to set up flags or color-codes ..

# Calculate a rolling mean value for the number of errors per CSC per dayobs, over last 10 reported dayobs (window)
# The need to have data available for a longer window here, means we needed to fetch more days in error codes above.
mean = dayerrs.T.rolling(window=10, min_periods=1, closed='left').mean().T

# And now we could compare any given day's errors to the average of the previous 10 days
diff = dayerrs - mean
diff = diff.fillna(0)

# And let's pick out the last last_n days to display to user
last_n = -30 #len(dayerrs) + 1
#diff = diff.iloc[:, last_n:]


# Then display the errors with a color-code by creating a styler based on the diff
style_df = (
        diff > 0.5        
).replace({
    True: 'background-color:pink',  # True Styles
    False: ''                      # False Styles
})

# And then apply this style to each value in each row, in the day errors values
for group in groups:
    group_topics_with_errors = [t for t in groups[group] if t in dayerrs.index.values]
    if len(group_topics_with_errors) > 0:
        subset = dayerrs.loc[group_topics_with_errors]
        # Calculate a rolling mean value for the number of errors per CSC per dayobs, over last 10 reported dayobs (window)
        # The need to have data available for a longer window here, means we needed to fetch more days in error codes above.
        mean = subset.T.rolling(window=10, min_periods=1, closed='left').mean().T
    
        # And now we could compare any given day's errors to the average of the previous 10 days
        diff = subset - mean
        diff = subset.fillna(0)
        diff = diff.iloc[:, last_n:] #last_n:]

        # Then display the errors with a color-code by creating a styler based on the diff
        style_df = (
                diff > 0.5        
        ).replace({
            True: 'background-color:pink',  # True Styles
            False: ''                      # False Styles
        })
        #display(subset.iloc[:, last_n:].style.apply(lambda _: style_df, axis=None).format(precision=0))
        display(subset.iloc[:, last_n:].style.apply(lambda _: style_df, axis=None).format(precision=0))

In [ ]:
# Can also make a stacked histogram of number of errors per CSC per night
plt.figure(figsize=(12, 6))

# Need to add final night for last (right) edge of last bin.
dayobs_range = np.arange(errors.day_obs_mjd.min(), errors.day_obs_mjd.max() + 2, 1)

# Show a subset of these days 
dayobs_range = dayobs_range[last_n-1:]

day_obs_labels = [Time(do_mjd, scale='utc', format='mjd').isot[0:10] for do_mjd in dayobs_range]

# Stacked histogram, so keep running bottom of where bottom of bar should be.
stack_bottom = np.zeros(len(dayobs_range)-1)
colors = glasbey 
#colors = tab20

i = 0
for ec in groups['Simonyi']:
    q = errors.query('component == @ec')
    if len(q) > 0:
        c, b = np.histogram(q['day_obs_mjd'], bins=dayobs_range)
        stack_bottom += c
        plt.bar(dayobs_range[0:-1], c, bottom=stack_bottom-c, label=ec, color=glasbey[i])
    i += 1

_ = plt.xticks(rotation=90, ticks=dayobs_range[:-1], labels=day_obs_labels[:-1])
_ = plt.xlabel("DayObs", fontsize='x-large')
_ = plt.ylabel("Number of errors", fontsize='x-large')
_ = plt.legend(loc=(1.01, 0.2))
#_ = plt.ylim(0, 200)
plt.grid(True, alpha=0.3)

In [ ]:
#simonyi_errs = errors.query("

#ve = pd.concat([visits, errors]).sort_index()
# find error -> visits
#error_to_image = np.where((ve[:-1]['salIndex'] > 0) and (ve[1:]['salIndex'] < 0)]


In [ ]:
# image -> error
im2err = np.where((ve[:-1]['salIndex'].values < 0 ) & (ve[1:]['salIndex'].values > 0))[0]
im2err = np.sort(np.concatenate([im2err, im2err+1]))
ve.iloc[im2err]